# Imports

In [1]:
from gurobipy import Model, GRB
from util import reachable_without_edge

# Problem Environment

In [2]:
generators = ["coal1", "coal2", "nuclear1", "hydro1", "hydro2", "coal3"]
loads = ["load1", "load2", "load3"]
zones = ["zone1", "zone2", "zone3"]

# Maximum MW each generator can supply
supply = {
    "coal1":100,
    "coal2":150,
    "nuclear1":200,
    "hydro1":150,
    "hydro2":150,
    "coal3":150
}

# USD/MW for each generator
cost = {
    "coal1":20,
    "coal2":35,
    "nuclear1":40,
    "hydro1":30,
    "hydro2":35,
    "coal3":25
}

# Demand in MW of each load
demand = {
    "load1":250,
    "load2":300,
    "load3":150
}

# Transmission limit across zones
transLimit = {
    ("zone1", "zone2"): 150,
    ("zone2", "zone3"): 100,
}

# Define the corresponding zone-generator relationships
generatorByZone = {
    "zone1": ["coal1", "coal2"],
    "zone2": ["nuclear1", "hydro1"],
    "zone3": ["hydro2", "coal3"]
}

# Define the corresponding zone-load relationships
loadByZone = {
    "zone1": ["load1"],
    "zone2": ["load2"],
    "zone3": ["load3"]
}

# Define the corresponding load-zone relationsips
zoneByLoad = {
    "load1": "zone1",
    "load2": "zone2",
    "load3": "zone3"
}

# Gurobi Setup and Optimization
## Setting up the necessary variables, constraints, and objective required for optimizing

In [ ]:
m = Model("LBMP")

x = m.addVars(generators, loads, name="x", lb=0) 

m.setObjective( sum(cost[g] * x[g,l] for g in generators for l in loads), GRB.MINIMIZE ) 

# Generator supply constraints 
for g in generators: 
    m.addConstr(sum(x[g,l] for l in loads) <= supply[g], name=f"supply_{g}") 
    
# Load demand constraints 
for l in loads: 
    m.addConstr(sum(x[g,l] for g in generators) == demand[l], name=f"demand_{l}") 


# create dictionary for each zone containing set
adj = {z: set() for z in zones}
#populate dictionary with the bordering zones for each zone
for (u,v) in transLimit.keys():
    adj[u].add(v)
    adj[v].add(u)

# define transmission bus constraints
for (zoneA,zoneB), cap in transLimit.items():
    # Side A contains all zones on one side of the partition
    T = reachable_without_edge(zoneA, zoneB, adj)
    # Side B contains the remaining zones
    Tbar = set(zones) - T

    # Find all generators in each partition
    gensA = [g for z in T for g in generatorByZone[z]]
    gensB = [g for z in Tbar for g in generatorByZone[z]]
    
    # Find all loads in each partition
    loadsA = [l for z in T for l in loadByZone[z]]
    loadsB = [l for z in Tbar for l in loadByZone[z]]

    # Compute the sums of energy flowing across the partitions
    flow_A_to_B = sum(x[g,l] for g in gensA for l in loadsB)
    flow_B_to_A = sum(x[g,l] for g in gensB for l in loadsA)

    # Combine directional energy sums and ensure the total is <= the
    # transmission cap for the corresponding bus
    m.addConstr(
        flow_A_to_B + flow_B_to_A <= cap,
        name=f"cut_{zoneA}_{zoneB}"
    )

m.optimize() 

if m.Status == GRB.OPTIMAL: 
    print("Optimal total cost:", m.ObjVal) 

    for g in generators: 
        for l in loads: 
            flow = x[g,l].X
            if flow > 0: 
                print(f"{g} -> {l}: {flow} MW")


# Price per Zone Calculation
## Determine if congestion occured, and assign prices to each zone accordingly

In [ ]:
# Get the MW usage of each generator
generatorUsage = {g: 0 for g in generators}
# Maximum cost of energy used in each zone
maxCostZone = {z: 0 for z in zones}

for g in generators: 
    for l in loads: 
        flow = x[g,l].X
        if flow > 0: 
            generatorUsage[g] += flow
            for z in zones:
                if(g in generatorByZone[z] and maxCostZone[z] < cost[g]):
                    maxCostZone[z] = max(cost[g], maxCostZone[z])

# Find minimum price of remaining energy across generators in all zones 
minPrice = -1
for z in zones:
    for g, e in generatorUsage.items():
        # Compute remaining energy at generator
        e = supply[g] - e
        if(e > 0 and g in generatorByZone[z]):
            if(cost[g] < minPrice or minPrice == -1):
                minPrice = cost[g]

# Calculate the marginal price (MP) and determine if congestion occurs
marginalPrice = -1
congestion = False
for g in generatorUsage.keys():
    # If no remaining energy in generators, all zones share the same price
    if(minPrice < 0):
        marginalPrice = max(marginalPrice, cost[g])
    # Find maximum price used by generators across all zones without surpassing
    # the minimum price of residual energy
    elif (cost[g] <= minPrice and cost[g] > marginalPrice):
        marginalPrice = cost[g]
    
    # If a cost is found above the minimum residual energy price, congestion 
    # has occured
    if (cost[g] > minPrice and minPrice > 0):
        congestion = True

# Set default price for each zone to the MP
zonePrice = {z: marginalPrice for z in zones}

# If congestion is present, set each zone's price to the maximum price used to supply energy
if (congestion):
    for g in generators: 
        for l in loads: 
            if (x[g,l].X > 0 and cost[g] > marginalPrice):
                zonePrice[zoneByLoad[l]] = max(zonePrice[zoneByLoad[l]], cost[g])

print("Price per Zone:")
for z, price in zonePrice.items():
    print(f"{z}:")
    print("\tMarginal Price:\t\t${marginalPrice:0.2f}".format(marginalPrice=marginalPrice))
    print("\tCongestion Cost: \t${congestionCost:0.2f}".format(congestionCost = price - marginalPrice))
    print("\tTotal Cost: \t\t${price:0.2f}".format(price = price))